In [1]:
from pytest import mark
from hypothesis import given, strategies as st
import ipytest
from pathlib import Path
from collections import defaultdict
import re
import regex
import timeit
from itertools import product
from functools import reduce
from operator import mul
ipytest.autoconfig()

In [2]:
input_path = Path(".") / "input"

In [3]:
files, data, solvers = {}, {}, {}

In [4]:
for f in input_path.iterdir():
    files[f.stem] = f.read_text()

In [5]:
class Solver:
    def __init__(self, data):
        raise NotImplementedError()
    def part1(self):
        raise NotImplementedError()
    def part2(self):
        raise NotImplementedError()

In [6]:
def solve_all(files):
    for key, contents in files.items():
        if f'Day{key}' in globals():
            day = globals()[f'Day{key}'](contents)
            try:
                part1 = day.part1()
            except:
                part1 = None
            try:
                part2 = day.part2()
            except:
                part2 = None
            print(f'''Day {key}
                Part 1: {part1}
                Part 2: {part2}''')

In [7]:
def name_to_number(name):
    values = {'one':'1', 'two':'2', 'three':'3', 'four':'4', 'five':'5', 'six':'6', 'seven':'7', 'eight':'8', 'nine':'9'}
    return values[name] if name in values else name

def line_to_calibration_value(line, pattern=r'(\d)'):
    nums = regex.findall(pattern, line, overlapped=True)
    return int(name_to_number(nums[0]) + name_to_number(nums[-1]))

class Day01(Solver):
    def __init__(self, data):
        self.data = data

    def part2(self):
        pattern = r'(\d|one|two|three|four|five|six|seven|eight|nine)'
        return sum(line_to_calibration_value(line, pattern) for line in self.data.splitlines())

    def part1(self):
        return sum(line_to_calibration_value(line) for line in self.data.splitlines())
    

In [8]:
%%ipytest -W "ignore:Module already imported"

day1_sample = """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
twone
eighthree
oneight
"""

day1_part2_interp = [29, 83, 13, 24, 42, 14, 76, 21, 83, 18]

@mark.parametrize("line,expected", zip(day1_sample.splitlines(), day1_part2_interp))
def test_day1_part2_parsing(line, expected):
    assert line_to_calibration_value(line, r'(\d|one|two|three|four|five|six|seven|eight|nine)') == expected

def test_day1_part2_sample():
    assert Day01(day1_sample).part2() == sum(day1_part2_interp)

def test_day1_part1():
    assert Day01(files['01']).part1() == 55130

def test_day1_part2():
    assert Day01(files['01']).part2() == 54985

.............                                                                                [100%]
13 passed in 0.09s


### Day 2

Each game is a series of draws and returns to the same bag. Determine which games are
possible given the following constraints:

> only 12 red cubes, 13 green cubes, and 14 blue cubes

Approach: For each game collect the max of each RGB cube and then filter by the above, sum the game ids.

In [9]:
def parse_game(game):
    id = int(re.search(r'(\d+)', game).groups(1)[0])
    rounds = game.split(':')[1].split(';')
    return (id, [round_to_dict(round) for round in rounds])

def round_to_dict(round):
    draw = defaultdict(int)
    pair = re.finditer(r'(\d+) (\w+)', round)
    for p in pair:
        count, color = int(p.group(1)), p.group(2)
        draw[color] = count
    return draw

def game_max(rounds):
    game = defaultdict(int)
    for round in rounds:
        for (color, count) in round.items():
            game[color] = max(game[color], count)
    return game

def possible(game):
    return game['red'] <= 12 and game['green'] <= 13 and game['blue'] <= 14

In [10]:
matches = parse_game(files['02'].splitlines()[0])

In [11]:
class Day02(Solver):
    def __init__(self, data):
        self.data = data
    def part1(self):
        games = [parse_game(game) for game in self.data.splitlines()]
        maxes = [(id, game_max(rounds)) for (id, rounds) in games]
        return sum(possible(game) * id for (id, game) in maxes)
    def part2(self):
        games = [parse_game(game) for game in self.data.splitlines()]
        maxes = [(id, game_max(rounds)) for (id, rounds) in games]
        p = 0
        for (id, m) in maxes:
            p += reduce(mul, m.values())
        return sum(reduce(mul, m.values()) for (_, m) in maxes)

In [12]:
%%ipytest -W "ignore:Module already imported"

day2_part1_sample = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""

day2_part1_parsed = [
    (1, [{'blue': 3, 'red': 4},
         {'red': 1, 'green': 2, 'blue': 6},
         {'green': 2}]),
    (2, [{'blue': 1, 'green': 2},
         {'green': 3, 'blue': 4, 'red': 1},
         {'green': 1, 'blue': 1}]),
    (3, [{'green': 8, 'blue': 6, 'red': 20},
         {'blue': 5, 'red': 4, 'green': 13},
         {'green': 5, 'red': 1}]),
    (4, [{'green': 1, 'red': 3, 'blue': 6},
         {'green': 3, 'red': 6},
         {'green': 3, 'blue': 15, 'red': 14}]),
    (5, [{'red': 6, 'blue': 1, 'green': 3},
         {'blue': 2, 'red': 1, 'green': 2}])
]

day2_part1_expected = 8 # 1, 2, 5 possible, 3 and 4 impossible

day2_part1_possible = [True, True, False, False, True]

day2_part2_values = [48, 12, 1560, 630, 36]

@mark.parametrize("game,expected", zip(day2_part1_sample.splitlines(), day2_part1_parsed))
def test_day2_parsing(game,expected):
    assert parse_game(game) == expected

@given(id=st.integers(min_value=0), draws=st.lists(st.dictionaries(keys=st.sampled_from(['red', 'green', 'blue']), values=st.integers(min_value=1,max_value=20), min_size=1), min_size=1))
def test_random_parsing(id, draws):
    """
    This is just fooling around. It'll generate a whole bunch of random sets, I construct the input and test
    that my parsing function produces the same result. It generates an int for the id and a list[dict] for
    the rounds. Each round is guaranteed to have at least one cube color and each game is guaranteed at
    least one round.
    """
    game = f'Game {id}: {"; ".join(", ".join(f"{val} {color}" for color,val in draw.items()) for draw in draws)}'
    parsed_id, parsed_draws = parse_game(game)
    assert parsed_id == id
    assert parsed_draws == draws

def test_day2_part1():
    day2 = Day02(day2_part1_sample)
    assert day2.part1() == 8

def test_day2_part1():
    day2 = Day02(day2_part1_sample)
    assert day2.part2() == sum(day2_part2_values)

@mark.parametrize("game,expected", zip(day2_part1_sample.splitlines(), day2_part1_possible))
def test_day2_part1_possible(game, expected):
    (id, rounds) = parse_game(game)
    maxes = game_max(rounds)
    assert possible(maxes) == expected

@mark.parametrize("game,expected", zip(day2_part1_sample.splitlines(), day2_part2_values))
def test_day2_part2_parts(game, expected):
    (id, rounds) = parse_game(game)
    maxes = game_max(rounds)
    print(maxes)
    assert reduce(mul, maxes.values()) == expected


.................                                                                            [100%]
17 passed in 0.30s


In [13]:
solve_all(files)

Day 01
                Part 1: 55130
                Part 2: 54985
Day 02
                Part 1: 3099
                Part 2: 72970
